In [1]:
import ezdxf
import numpy as np

In [24]:
# Create a new DXF document.
doc = ezdxf.new(dxfversion='R2010')

# Create new table entries (layers, linetypes, text styles, ...).
layer = doc.layers.new('main', dxfattribs={'lineweight': 0.0254})
layer.rgb = (255, 0, 0)

# DXF entities (LINE, TEXT, ...) reside in a layout (modelspace, 
# paperspace layout or block definition).  
msp = doc.modelspace()

In [25]:
# Define constants
width = 160
gripper_height = 50
gripper_thickness = 20
handle_height = 200
handle_thickness = 15
handle_hole_diameter = 10
handle_hole_distance_from_top = 10
screw_diameter = 6.5

In [26]:
def add_circle(center, radius, start_angle=0, end_angle=360, is_counter_clockwise=True):
    num_points = 15
    start_angle *= np.pi/180
    end_angle *= np.pi/180
    if is_counter_clockwise:
        theta = np.arange(start_angle, end_angle, abs(end_angle-start_angle)/num_points)
    else:
        theta = np.arange(end_angle, 2*np.pi + start_angle, abs(2*np.pi + start_angle - end_angle)/num_points)
    for n in range(num_points - 1):
        start_pos = (center[0] + radius*np.cos(theta[n]), center[1] + radius*np.sin(theta[n]))
        end_pos = (center[0] + radius*np.cos(theta[n+1]), center[1] + radius*np.sin(theta[n+1]))
        msp.add_line(start_pos, end_pos)     
    
    start_pos = (center[0] + radius*np.cos(theta[-1]), center[1] + radius*np.sin(theta[-1]))
    if is_counter_clockwise:
        end_pos = (center[0] + radius*np.cos(end_angle), center[1] + radius*np.sin(end_angle))
    else:
        end_pos = (center[0] + radius*np.cos(2*np.pi + start_angle), center[1] + radius*np.sin(2*np.pi + start_angle))
    msp.add_line(start_pos, end_pos)     
    

In [27]:
def add_outline(pts):
    for i in range(len(pts) - 1):
        msp.add_line(pts[i], pts[i+1])

In [28]:
add_outline([(0, 0),
            (width, 0),
            (width, gripper_height),
            (width/2 + handle_thickness/2, gripper_height),
            (width/2 + handle_thickness/2, gripper_height + handle_height),
            (width/2 - handle_thickness/2, gripper_height + handle_height),
            (width/2 - handle_thickness/2, gripper_height),
            (0, gripper_height),
            (0, 0)])

add_outline([(gripper_thickness, gripper_thickness),
            (width - gripper_thickness, gripper_thickness),
            (width - gripper_thickness, gripper_height - gripper_thickness),
            (gripper_thickness, gripper_height - gripper_thickness),
            (gripper_thickness, gripper_thickness)])

add_circle((width/2, gripper_height + handle_height - handle_hole_distance_from_top), handle_hole_diameter/2)
add_circle((gripper_thickness/2, gripper_thickness/2), screw_diameter/2)
add_circle((width - gripper_thickness/2, gripper_thickness/2), screw_diameter/2)

In [29]:
# Save DXF document.
doc.saveas('20201129_leverarm.dxf')